In [56]:
import pandas as pd
import numpy as np

artist_data = pd.read_csv('datasets/ydata-ymusic-user-artist-rating.csv')
songs_data = pd.read_csv('datasets/ydata-ymusic-user-song-rating.csv')
artist_name_data = pd.read_csv('datasets/ydata-ymusic-artist-names.csv')

In [57]:
#Data preprocessing
def data_preprocessing(artist_data, groupByVar, indexId):
    
    #to show popular artist
    avg_rating = pd.DataFrame(artist_data.groupby(groupByVar)['rating'].mean().reset_index())
    avg_rating.sort_values('rating', ascending = False).head(20)
    
    #creating pivot table
    ratedBygroupByVar = pd.pivot_table(data=artist_data, values='rating', index=indexId, columns=groupByVar)
    
    return ratedBygroupByVar

In [58]:
def data_cleaning(recommended_data):
    
     #data cleaning and sorting
        
    recommended_corr = pd.DataFrame(recommended_data, columns=['Correlation'])
    recommended_corr.dropna(inplace=True)
    recommended_corr = recommended_corr.sort_values('Correlation', ascending=False).reset_index()
    
    return recommended_corr

In [64]:
def get_top_n_artist(recommended_corr, groupByVar, n, filteringFrom):
    
    #get top n artist list and artist names
    recommended_list = recommended_corr.head(n)[groupByVar].tolist()
    filter_names = filteringFrom[filteringFrom[groupByVar].isin(recommended_list)]
    
    return filter_names

In [69]:
def collaborative_filtering(data, groupByVar, indexId, Id, n, filteringFrom):
    
    #calling data preprocessing
    rating = data_preprocessing_general(data, groupByVar, indexId)
    
    #artist_id 
    #see groupByVar rated by indexId
    groupByVar_indexId_rating = rating[Id]
    
    #applying correlation for the entire user rating dataframe
    #Collaborative Filtering Song Recommendation
    #Pearson Correlation
    recommended_data = rating.corrwith(groupByVar_indexId_rating)
    
    
    #data cleaning and sorting
    recommended_corr = data_cleaning_general(recommended_data)
    
    #get top n artist list and artist names
    if(filteringFrom is None):
        recommended_items = recommended_corr.head(n)[groupByVar].tolist()
    else:
        recommended_items = get_top_n_artist_general(recommended_corr, groupByVar, n, filteringFrom)
    
    return recommended_items

In [70]:
recommended_artist_names = collaborative_filtering(artist_data,'artist_id', 'user_id', 1058037, 5, artist_name_data)
recommended_artist_names

IndexError: Inconsistent shape between the condition and the input (got (14158, 1) and (14158,))

In [62]:
recommended_songs_Ids = collaborative_filtering(songs_data, 'song_id', 'user_id', 90134, 5, None)

In [63]:
recommended_songs_Ids

[44204, 45036, 36829, 21306, 92419]

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


# calculate the correlation matrix
corr_matrix = artist_data.pivot_table(index='user_id', columns='artist_id', values='rating').corr()

# visualize the correlation matrix using a heatmap
plt.figure(figsize=(15,15))
plt.imshow(corr_matrix, cmap='coolwarm')
plt.colorbar()
plt.show()